# Test of CSV file produced using ROOT converter

In this notebook I'll test the macro, written in python, used for converting a ROOT structure (Tree) into a CSV file for ML analysis.

The CSV is located inside the directory output, created by the python macro **examplemacro.py**:

In [1]:
!ls /Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/

Jupyter_Notebooks_miscellaneous exampleMacro.py
MuonTree.root                   interface
README.md                       output
config                          results
exampleMacro                    tdrstyle.C
exampleMacro.C


In [2]:
%cd /Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output
!head -5 output_bxcut.csv

/Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output







To analyze it, we need the ROOT python module and pandas for the CSV analysis:

In [3]:
from ROOT import TCanvas,TH1F
import pandas as pd
import numpy as np

Welcome to JupyROOT 6.12/04


Now we need to read the CSV:

In [4]:
df2 = pd.read_csv('./output_bxcut.csv')

This creates a pandas db that can be displayed showing its content:

In [5]:
a = 1
new_column = []
row_list=[]
final_row=[]
for column in df2.columns.values.tolist():
    if column==df2.columns.values.tolist()[0]:
        new_column.append(column)
        new_column.append("n_Primitive")
        continue
    for count in range(1,df2["Event"].value_counts().max()+1):
        new_column.append(column + str(count))

df = pd.DataFrame(columns=new_column)
for index, row in df2.iterrows():
    if (row["Event"] == a):
        row_list.append(row.tolist())
        continue
    elif(row["Event"] != a):
        for i in range(0,len(df2.columns)):
            if i==0:
                final_row.append(row_list[0][0])
                final_row.append(len(row_list))
                continue
            for j in range(0,len(row_list)):
                final_row.append(row_list[j][i])
                if j == len(row_list)-1:   
                    if len(row_list) != df2["Event"].value_counts().max():
                        final_row += [np.nan]*(df2["Event"].value_counts().max()-len(row_list))
        a = row["Event"]
        df.loc[row["Event"]-1] = final_row
        del row_list[:]
        del final_row[:]
        row_list.append(row.tolist())

for i in range(0,len(df2.columns)):
    if i==0:
        final_row.append(row_list[0][0])
        final_row.append(len(row_list))
        continue
    for j in range(0,len(row_list)):
        final_row.append(row_list[j][i])
        if j == len(row_list)-1:   
            if len(row_list) != df2["Event"].value_counts().max():
                final_row += [np.nan]*(df2["Event"].value_counts().max()-len(row_list))
df.loc[row["Event"]] = final_row

In [6]:
df = df.rename(columns = {"genParticle.pt1":"genParticle.pt"})

In [7]:
for i in range(2,df2["Event"].value_counts().max()+1):
    title = "genParticle.pt" + str(i)
    df = df.drop(title,1)

Then put the organized table inside a csv file.

In [10]:
df.to_csv("/Users/tommaso/TESI_MAGISTRALE/MuonPOGAnalysisTemplate/output/bxcut_org.csv",na_rep=0,index=False)